# Домашнее задание 4. Конструирование текстовых признаков из твитов пользователей

## Сбор данных

Первый этап - сбор твитов пользователей. Необходимо подключаться к Twitter API и запрашивать твиты по id пользователя. 
Подключение к API подробно описано в ДЗ 1.

In [1]:
import twitter

CONSUMER_KEY = ""
CONSUMER_SECRET = ""

ACCESS_TOKEN_KEY = ""
ACCESS_TOKEN_SECRET = ""

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET)

Для получения твитов пользователя может быть использован метод GetUserTimeline из библиотеки python-twitter. Он позволяет получить не более 200 твитов пользователя.

Метод имеет ограничение по количеству запросов в секунду. Для получения информации о промежутке времени, которое необходимо подождать для повторного обращения к API может быть использован метод `GetSleepTime`. Для получения информации об ограничениях запросов с помощью метода `GetUserTimeLine` необходимо вызывать `GetSleepTime` с параметром "statuses/user_timeline".

Метод GetUserTimeline возвращает объекты типа Status. У этих объектов есть метод AsDict, который позволяет представить твит в виде словаря.

Id пользователей необходимо считать из файла, как было сделано в ДЗ 1.

Необходимо реализовать функцию `get_user_tweets(user_id)`. Входной параметр - id пользователя из файла. Возвращаемое значение - массив твитов пользователя, где каждый твит представлен в виде словаря. Предполагается, что информация о пользователе содержится в твитах, которые пользователь написал сам. Это означает, что можно попробовать отфильтровать ответы другим пользователям, ссылки и ретвиты, а так же картинки и видео, так как наша цель - найти текстовую информацию.

In [2]:
import time

def get_user_tweets(user_id):
    print "Processing user " + str(user_id)
    all_tweets = []
    statuses = None
    while statuses is None:
        try:
            statuses = api.GetUserTimeline(user_id, count=200, include_rts=False,
                                                   trim_user=True, exclude_replies=True)
        except Exception as e:
            print e
            sleep_time = api.GetSleepTime("statuses/user_timeline")
            if sleep_time > 0:
                print "Going to sleep " + str(sleep_time) + " seconds"
                time.sleep(sleep_time)
            else:       
                print "PROBLEM with " + str(user_id)
                return []
        
    all_tweets = statuses
    all_tweets = [tweet.AsDict()['text'] for tweet in all_tweets]
    print "Tweets for " + str(user_id) + " is " + str(len(all_tweets))
    return all_tweets

## Разбор текста твита

Обработка текста предполагает разбиение текста на отдельные элементы - параграфы, предложения, слова. Мы будем преобразовывать текст твита к словам. Для этого текст необходимо разбить на слова. Сделать это можно, например, с помощью регулярного выражения.

Необходимо реализовать функцию, `get_words(text)`. Входной параметр - строка с текстом. Возвращаемое значение - массив строк (слов). Обратите внимание, что нужно учесть возможное наличие пунктуации и выделить по возможности только слова. 

In [3]:
from nltk.tokenize import TweetTokenizer
import nltk.tokenize.casual
import string
import re
import unicodedata
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [33]:
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
punctuation = string.punctuation

REGEXPS = (
    nltk.tokenize.casual.URLS,
    # ASCII Emoticons
    nltk.tokenize.casual.EMOTICONS
    ,
    # HTML tags:
    r"""<[^>\s]+>"""
    ,
    # ASCII Arrows
    r"""[\-]+>|<[\-]+"""
    ,
    # Ellipsis dots.
    r"""(?:\.(?:\s*\.){1,})"""
    ,
    # Numbers, including fractions, decimals.
    r"""(?:[+\-]?\d+([,/.:-]\d+[+\-]?)?)"""
    ,
    #Twitter hashtags
    r"""(?:\#+[\w_]+[\w\'_\-]*[\w_]+)"""
)

PURE_RE = re.compile(r"""(%s)""" % "|".join(REGEXPS), re.VERBOSE | re.I | re.UNICODE)

In [34]:
def get_words(text):
    """returns list of words"""
    # your code here
    words = [PURE_RE.sub("", token) for token in tokenizer.tokenize(text) if token not in punctuation]
    return [word for word in words if word]

Далее полученные слова необходимо привести к нормальной форме. То есть привести их к форме единственного числа настоящего времени и пр. Сделать это можно с помощью библиотеки nltk. Информацию по загрузке, установке библиотеки и примерах использования можно найти на сайте http://www.nltk.org/

Для загрузки всех необходимых словарей можно воспользоваться методом download из библиотеки nltk.

In [6]:
import nltk
nltk.download()

showing info http://www.nltk.org/nltk_data/


True

Для дальнейшей обработки слова должны быть приведены к нижнему регистру. 

Для приведения к нормальной форме можно использовать `WordNetLemmatizer` из библиотеки nltk. У этого класса есть метод `lemmatize`.

Также необходимо убрать из текста так называемые стоп-слова. Это часто используемые слова, не несущие смысловой нагрузки для наших задач. Сделать это можно с помощью `stopwords` из nltk.corpus

Необходимо реализовать функцию `get_tokens(words)`. Входной параметр - массив слов. Возвращаемое значение - массив токенов.

In [35]:
def get_tokens(words):
    """returns list of tokens"""
    # your code here
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word.lower()) for word in words]
    tokens = [word for word in lemmatized_words if word not in stopwords.words('english')]
    return tokens

Необходимо реализовать функцию `get_tweet_tokens(tweet)`. Входной параметр - текст твита. Возвращаемое значение -- токены твита. 

In [36]:
def get_tweet_tokens(tweet):
    # your code here
    words = get_words(tweet)
    tokens = get_tokens(words)
    return tokens

Необходимо реализовать функцию `collect_users_tokens()`. Функция должна сконструировать матрицу признаков пользователей. В этой матрице строка - пользователь. Столбец - токен. На пересечении - сколько раз токен встречается у пользователя.
Для построения матрицы можно использовать `DictVectorizer` из `sklearn.feature_extraction`.

In [37]:
def collect_users_tokens(df_users, tweet_collection):
    """returns users list and list of user dicts. Each dict contains frequence of user tokens"""
    # your code here
    users = []
    user_tokens = []
    
    tokens = {}
    for user in df_users["twitter_id"]:
        users.append(str(user))
        for tweet in tweet_collection[str(user)]:
            tweet_tokens = get_tweet_tokens(tweet)
            for token in tweet_tokens:
                if token in tokens.keys():
                    tokens[token] += 1
                else:
                    tokens[token] = 1
        user_tokens.append(tokens)
        tokens = {}
        
    return users, user_tokens

In [38]:
import pandas as pd
from collections import defaultdict
import json
from sklearn.feature_extraction import DictVectorizer


TRAINING_SET_URL = "twitter_train.csv"
df_users = pd.read_csv(TRAINING_SET_URL, sep=",", header=0, names=["twitter_id", "is_1", "is_2", "is_3"])
df_users.head()

,twitter_id,is_1,is_2,is_3
0,66412773,0,0,1
1,10143902,0,0,1
2,73701917,0,0,1
3,82209363,0,0,1
4,47063951,0,0,1


In [39]:
import numpy as np

users = df_users['twitter_id'].values
user_chunks = np.split(users, 100)

In [ ]:
user_tweets = {}
chunk_num = 15

while chunk_num < len(user_chunks):
    users = user_chunks[chunk_num]
    print "Proccessing chunk ", chunk_num
    for twitter_id in users:
        user_tweets[twitter_id] = get_user_tweets(twitter_id)
    
    f = open("tweets_" + str(chunk_num) + ".txt", "w")
    json.dump(user_tweets, f)
    user_tweets = {}
    f.close()
    chunk_num += 1

In [40]:
tweet_collection = {}

#Merge chunks with user tweets
for i in xrange(100):
    f = open("short_tweets/tweets_" + str(i) + ".txt")
    users = json.load(f)
    for key in users.keys():
        tweet_collection[key] = users[key]
    f.close()

In [41]:
users, users_tokens = collect_users_tokens(df_users, tweet_collection)
v = DictVectorizer()
vs = v.fit_transform(users_tokens)

In [42]:
import numpy as np
np.savez("out_4.dat", data=vs, users=users, users_tokens=users_tokens)

Далее для получения представления о полученной информацию о токенах предлагается отобразить ее в виде облака тэгов. [Подсказка](http://anokhin.github.io/img/tag_cloud.png). 

In [43]:
import wordcloud

In [56]:
def draw_tag_cloud(v, vs):
    """Draws tag cloud of found tokens"""
    # your code here
    filename = "cloud.png"
    frequency = np.asarray(vs.sum(axis=0))
    frequency = frequency.reshape(frequency.shape[1])
    n = np.sum(frequency)
    frequency = frequency / n
    words = v.get_feature_names()
    
    freqs = []
    for i, word in enumerate(words):
        freqs.append((word, frequency[i]))
    
    freqs.sort(key=lambda x: x[1], reverse=True)
    
    cloud = wordcloud.WordCloud(width=1920, height=1080, max_words=1000)
    cloud.fit_words(freqs)
    cloud.to_file(filename)
    return

In [57]:
draw_tag_cloud(v, vs)